In [22]:
import pandas as pd
data = pd.read_csv('Load Data in kW.csv')
data.head()

,Date,Time,Load (kW),Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,01.09.2018,0:00,5551.82208,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1:00,4983.17184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2:00,4888.39680,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,3:00,5072.95872,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,4:00,5196.25980,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
day_1 = data.iloc[0:-24, 2]
day_2 = data.iloc[24:, 2]

In [18]:
day_2 = day_2.reset_index()
day_2 = day_2['Load (kW)'] 

In [4]:
df = pd.concat([day_1, day_2], axis = 1)

In [24]:
df.columns = ['Day_1', 'Day_2']
df.head()

,Day_1,Day_2
0,5551.82208,4931.26380
1,4983.17184,4775.53968
2,4888.39680,4713.81120
3,5072.95872,4689.02628
4,5196.25980,4844.75040


In [6]:
normalized_df = (df-df.mean())/df.std()
normalized_df.head()

,Day_1,Day_2
0,-0.454862,-1.026890
1,-0.988072,-1.172657
2,-1.076940,-1.230438
3,-0.903881,-1.253638
4,-0.788264,-1.107871


In [21]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(normalized_df.Day_1, normalized_df.Day_2, test_size = 0.10, random_state = 42)

In [8]:
m = 1 #Initial value of slope
c = -1 #Initial value of intercept
lr = 0.01 #Learning Rate
delta_m = 1 #Initialising Δm
delta_c = 1 #Initialising Δc
v_m = 0
v_c = 0
lam = 0.9
max_iters = 100 #Maximum number of iterations  
iters_count = 0 #Counting Iterations


def deriv(m_f, c_f, x, y):
  m_deriv = -1*(y-m_f*x-c_f)*x
  c_deriv = -1*(y-m_f*x-c_f)
  return m_deriv, c_deriv  


while iters_count < max_iters:
  for i in range(x_train.shape[0]):
    delta_m, delta_c = deriv(m, c, x_train.iloc[i], y_train.iloc[i])
    v_m = lam * v_m - lr * delta_m
    v_c = lam * v_c - lr * delta_c
    m += v_m
    c += v_c
  iters_count += 1

In [9]:
print(f"\nThe local minima occurs at: {m}, {c}")


The local minima occurs at: 0.8885266666378754, -0.015125603603097632


In [25]:
import numpy as np

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)

In [26]:
y_pred_train = []
for i in x_train:
  y_p_tr = (m * i) + c
  y_pred_train.append(y_p_tr)
y_pred_train = np.array(y_pred_train)

In [27]:
y_pred_test = []
for i in x_test:
  y_p_te = (m * i) + c
  y_pred_test.append(y_p_te)
y_pred_test = np.array(y_pred_test)

In [28]:
import math
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import mean_absolute_error


#Training Accuracies
mse = math.sqrt(mean_squared_error(y_train, y_pred_train)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_train, y_pred_train)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_train, y_pred_train)
print('Mean absolute error', mae)

Root mean square error 0.6888134336509062
Mean square error 0.4744639463779513
Mean absolute error 0.4922787663231915


In [29]:
#Testing Accuracies
mse = math.sqrt(mean_squared_error(y_test, y_pred_test)) 
print('Root mean square error', mse) 
mse = (mean_squared_error(y_test, y_pred_test)) 
print('Mean square error', mse) 
mae=mean_absolute_error(y_test, y_pred_test)
print('Mean absolute error', mae)

Root mean square error 0.6650642307346853
Mean square error 0.4423104310027188
Mean absolute error 0.48048030390084917


In [30]:
hour = input("Enter an hour(0-23) of 1st December, 2018 to predict the load")
index = 2137 + int(hour)
x = normalized_df.iloc[index, 1]
normalised_output = m * x + c
output = (normalised_output * df.std()) + df.mean()
print(f"Predicted Load (kW) at {hour} hours on 1st December, 2018: {output[0]}")

Enter an hour(0-23) of 1st December, 2018 to predict the load5
Predicted Load (kW) at 5 hours on 1st December, 2018: 5129.989530695806
